In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models,layers

In [19]:
train_folder="C:/Users/DELL/Desktop/GROUP_PROGECT/Skin_Conditions"

val_folder="C:/Users/DELL/Desktop/GROUP_PROGECT/Skin_Conditions"

In [20]:
train_datagen=ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen=ImageDataGenerator(rescale=0.1/255)

In [21]:
trainGen=train_datagen.flow_from_directory(
    train_folder,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

valGen=val_datagen.flow_from_directory(
    val_folder,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

Found 2394 images belonging to 6 classes.
Found 2394 images belonging to 6 classes.


In [28]:
classNames=list(trainGen.class_indices.keys())

classNames

['Acne', 'Carcinoma', 'Eczema', 'Keratosis', 'Milia', 'Rosacea']

In [34]:
from tensorflow.keras import layers, models

base_model = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(trainGen.num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [36]:
# Train the model
history = model.fit(
    trainGen,
    epochs=10,  
    validation_data=valGen,
    steps_per_epoch=trainGen.samples,
    validation_steps=valGen.samples
)


Epoch 1/10


c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  75/2394 ━━━━━━━━━━━━━━━━━━━━ 4:29:58 7s/step - accuracy: 0.2657 - loss: 1.9435

c:\Users\DELL\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


2394/2394 ━━━━━━━━━━━━━━━━━━━━ 1076s 433ms/step - accuracy: 0.3713 - loss: 1.6421 - val_accuracy: 0.5088 - val_loss: 1.4415
Epoch 2/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 1009s 416ms/step - accuracy: 0.6055 - loss: 1.0855 - val_accuracy: 0.5343 - val_loss: 1.3366
Epoch 3/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 1022s 422ms/step - accuracy: 0.6624 - loss: 0.9266 - val_accuracy: 0.5660 - val_loss: 1.2689
Epoch 4/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 939s 388ms/step - accuracy: 0.6970 - loss: 0.8432 - val_accuracy: 0.5777 - val_loss: 1.2129
Epoch 5/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 931s 384ms/step - accuracy: 0.7289 - loss: 0.7716 - val_accuracy: 0.5865 - val_loss: 1.1829
Epoch 6/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 925s 382ms/step - accuracy: 0.7353 - loss: 0.7292 - val_accuracy: 0.5886 - val_loss: 1.1547
Epoch 7/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 926s 382ms/step - accuracy: 0.7535 - loss: 0.6948 - val_accuracy: 0.6019 - val_loss: 1.1260
Epoch 8/10
2394/2394 ━━━━━━━━━━━━━━━━━━━━ 925s 382ms/step - accuracy: 

In [38]:
model.save("resnet152v2.h5")

In [75]:
imagePath = 'C:/Users/DELL/Desktop/GROUP_PROGECT/Skin_Conditions/Acne/Acne_16.jpg'

image=tf.keras.preprocessing.image.load_img(imagePath,target_size=(224,224))
img_array=tf.keras.preprocessing.image.img_to_array(image)
img_exp=tf.expand_dims(img_array,axis=0)
img=img_exp/255

predictions=model.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step


In [76]:
predictions

array([[0.34680167, 0.08183535, 0.12697849, 0.03975961, 0.10581183,
        0.29881305]], dtype=float32)

In [77]:
import numpy as np

In [78]:
pred=np.argmax(predictions,axis=1)
pred

array([0], dtype=int64)

In [65]:
for name,index in enumerate(trainGen.class_indices):
    if name==pred[0]:
        pred2=index
        break

pred2

'Eczema'